In [1]:
import dlib        
import numpy as np 
import cv2          
import os          
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.linear_model import LogisticRegressionCV

In [2]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('./shape_predictor_68_face_landmarks.dat')

In [3]:
lip_data=[]
nothing_number=[]
for i in range (5000):
    img = cv2.imread('../datasets/celeba/img/'+str(i)+'.jpg')
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    positions_68_arr = []
    faces = detector(img_gray, 0)
    if len(faces) !=0:
        landmarks = np.matrix([[p.x, p.y] for p in predictor(img, faces[0]).parts()])

        for idx, point in enumerate(landmarks):
        #coordinates of 68 points
            pos = (point[0, 0], point[0, 1])
            positions_68_arr.append(pos)

        positions_lip_arr = []
        #coordinates of 49-68 points which represents mouth
        for i in range(48, 68):
            positions_lip_arr.append(positions_68_arr[i][0])
            positions_lip_arr.append(positions_68_arr[i][1])
        lip_data.append(positions_lip_arr)
    else:
        nothing_number.append(i)
        continue

In [5]:
import pandas as pd
lip_data_dt = pd.DataFrame(lip_data)
lip_data_dt.to_csv('lip.csv')

In [4]:
label=pd.read_table('../datasets/celeba/labels.csv')

In [6]:
# Split the data
x = lip_data
y = label['smiling']

In [7]:
for i in range(len(nothing_number)):
    del y[nothing_number[i]]

# Split the data into training and testing(75% training and 25% testing data)
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state=0)# Pre-process data
from sklearn.preprocessing import StandardScaler
x_train = StandardScaler().fit_transform(x_train)
x_test = StandardScaler().fit_transform(x_test)

In [9]:
import pandas as pd
y_dt = pd.DataFrame(y)
y.to_csv('true_label.csv')

In [8]:
logreg = LogisticRegression(solver='lbfgs',max_iter=1000)
logreg.fit(x_train, y_train)
y_pred= logreg.predict(x_test)
print('Accuracy on train set:'+str(logreg.score(x_train,y_train)))
print('Accuracy on test set: '+str(accuracy_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))#text report showing the main classification metrics

Accuracy on train set:0.8847634763476347
Accuracy on test set: 0.8821104699093157
              precision    recall  f1-score   support

          -1       0.87      0.90      0.88       594
           1       0.90      0.87      0.88       619

    accuracy                           0.88      1213
   macro avg       0.88      0.88      0.88      1213
weighted avg       0.88      0.88      0.88      1213



In [10]:
import pickle

#save model
pickle.dump(logreg,open("LogisticRegression_smiling_split_dlib_lip.dat","wb"))  

# load model
loaded_model = pickle.load(open("LogisticRegression_smiling_split_dlib_lip.dat","rb"))

y_pred=loaded_model.predict(x_test)